# Pectoralis Segmentation: Convert to h5

In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
import SimpleITK as sitk
# %matplotlib inline
import h5py
import progressbar
import SimpleITK as sitk
import matplotlib.pyplot as plt
import csv
import pandas as pd

In [ ]:
def get_keys(f):
    return [key for key in f.keys()]

In [ ]:
def get_keys(h5_path):
    with h5py.File(h5_path, 'r') as f:
        keys = [key for key in f.keys()]
    return keys

In [ ]:
def create_newH5(output_path):
    with h5py.File(output_path, 'w') as f:
        pass
    
    return None
    
def create_dset(h5_path, dataKey, dataShape, dim_maxsize, dtype, data):
    with h5py.File(h5_path, 'r+') as f:
        maxshape = list(dataShape[:])
        maxshape[dim_maxsize] = None
        maxshape = tuple(maxshape)
        
        chunks = list(dataShape[:])
        chunks[dim_maxsize] = 1
        chunks = tuple(chunks)
        
        data_orig = f.create_dataset(dataKey, dataShape, dtype=dtype, maxshape=maxshape, data=data, chunks=chunks)
    
    return None

def create_dset_no_data(h5_path, dataKey, dataShape, dim_maxsize, dtype):
    with h5py.File(h5_path, 'r+') as f:
        maxshape = list(dataShape[:])
        maxshape[dim_maxsize] = None
        maxshape = tuple(maxshape)
        
        chunks = list(dataShape[:])
        chunks[dim_maxsize] = 1
        chunks = tuple(chunks)
        
        data_orig = f.create_dataset(dataKey, dataShape, dtype=dtype, maxshape=maxshape, chunks=chunks)
    
    return None


def add_to_dset(h5_path, dataKey, idim_to_add, new_data):
    with h5py.File(h5_path, 'r+') as f:
        ## Comprobar que el dset existe con la funcion keys
        dset = f[dataKey]
        dset_shape = list(dset.shape)
        new_dataShape = list(new_data.shape)
        
#         print('previos shape: ', dset_shape)
#         print('previos shape: ', new_dataShape)
        a = dset_shape[idim_to_add]
        b = new_dataShape[idim_to_add]
        dset_shape[idim_to_add] = a + b
    
#         print('new final shape: ', dset_shape)
        dset.resize(tuple(dset_shape))
        
        if idim_to_add == 0:
            dset[a:] = new_data
    
    return None
        
def modify_dset(h5_path, dataKey, indexes, data):
    with h5py.File(h5_path, 'r+') as f:
        ## Comprobar que el dset existe con la funcion keys
        dset = f[dataKey]
        dset[indexes] = data
    return None

def delete_dset(h5_path, dataKey):
    with h5py.File(h5_path, 'r+') as f:
        del f[dataKey]
    return None


def get_shape_dset(h5_path, dataKey):
    with h5py.File(h5_path, 'r') as f:
        shape = f[dataKey].shape
    return shape


def get_dset_idx(h5_path, dataKey, idx):
    with h5py.File(h5_path, 'r') as f:
        data = f[dataKey][idx]
    return data

----------

----

## Get Info from h5

In [ ]:
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'
folder_daneses = '/data/DNNData/la660/Data/DataSet.h5'

In [ ]:
keys = get_keys(h5_path)

for key in keys:
    print(key, ': \t\t', get_shape_dset(h5_path, key))

-------

-----

## Adding H5PY DataSets

#### Creates new H5 file

In [ ]:
## Create H5
output_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'
# create_newH5(output_path)

#### Add atributes

#### Raw Images data set

In [ ]:
folder = '/Users/rmoreta/Documents/BWH Estancia 2016/ProjectData_clean/'
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'

widgets = ["Saving to h5: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=7*2, widgets=widgets).start()

dataKey = 'images_raw'
h5_keys = get_keys(h5_path)
if dataKey in h5_keys:
    print('Key already existed. Now is deleted.', dataKey)
    delete_dset(h5_path, dataKey)

num_files = range(7)
for i in num_files:
    # Reading Slices
    file_name_CT = 'Cont_' + str(i+1) + '_clean.nrrd'
    image_nrrd = sitk.ReadImage(folder + file_name_CT)
    CTSlices = sitk.GetArrayFromImage(image_nrrd)
    dataShape = CTSlices.shape
    print(file_name_CT, 'shape: ', dataShape)
    pbar.update((i*2))
    
    if i == num_files[0]:
        ## Create dataset in H5
        create_dset(h5_path, dataKey, dataShape, 0, 'f', CTSlices)
        
    else:
        ## Add data to a extinting dataset
        idim_to_add = 0
        add_to_dset(h5_path, dataKey, idim_to_add, CTSlices)
        
    pbar.update(i*2+1)
pbar.finish()

#### Raw Labels

In [ ]:
folder = '/Users/rmoreta/Documents/BWH Estancia 2016/ProjectData_clean/'
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'

dataKey = 'labels_raw'
h5_keys = get_keys(h5_path)
if dataKey in h5_keys:
    print('Key already existed. Now is deleted.', dataKey)
    delete_dset(h5_path, dataKey)
    
widgets = ["Saving to h5: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=7*2, widgets=widgets).start()

num_files = range(7)
for i in num_files:
    # Reading Slices
    file_name_label = 'Cont_lm_' + str(i+1) + '_clean.nrrd'
    image_nrrd = sitk.ReadImage(folder + file_name_label)
    labels = sitk.GetArrayFromImage(image_nrrd)
    dataShape = labels.shape
    print(file_name_label, 'shape: ', dataShape)
    pbar.update((i*2))
    
    if i == num_files[0]:
        ## Create dataset in H5
        create_dset(h5_path, dataKey, dataShape, 0, 'f', labels)
        
    else:
        ## Add data to a extinting dataset
        idim_to_add = 0
        add_to_dset(h5_path, dataKey, idim_to_add, labels)
    pbar.update(i*2+1)
pbar.finish()

#### Normalized Images

In [ ]:
def norm_images(data):
    norm_data = data.astype(np.float32)
    norm_data[norm_data < -1024] = -1024
    norm_data[norm_data > 1500] = 1500
    norm_data = (norm_data + 1024) / (1500 + 1024) - 0.5

    return norm_data

In [ ]:
folder = '/Users/rmoreta/Documents/BWH Estancia 2016/ProjectData_clean/'
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'

dataKey = 'images_norm'
h5_keys = get_keys(h5_path)
if dataKey in h5_keys:
    print('Key already existed. Now is deleted.')
    delete_dset(h5_path, dataKey)

dset_size = get_shape_dset(h5_path, 'images_raw')
im_size = (0, dset_size[1], dset_size[2])
create_dset_no_data(h5_path, dataKey, im_size, 0, 'f')
len_dset = dset_size[0]
    
widgets = ["Saving to h5: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len_dset, widgets=widgets).start()

buff_size = 1000
for i in np.arange(0, len_dset, buff_size):
    if i+buff_size > len_dset:
        idx = range(i, len_dset)
    else:
        idx = range(i, i+buff_size)
        
    data = get_dset_idx(h5_path, 'images_raw', idx)
    data_norm = norm_images(data)
    add_to_dset(h5_path, dataKey, 0, data_norm)
        
    pbar.update(i)
pbar.finish()

#### Labels n classes

In [ ]:
def separate_labels(labels, num_classes):
    ERROR = True
    if num_classes == 2:
        labels[(labels!=13335)&(labels!=13336)&(labels!=13591)&(labels!=13592)&(labels!=17943)&(labels!=17944)] = 0
        labels[(labels==13335)] = 1 # Left Minor Pectoral
        labels[(labels==13336)] = 1 # Right Minor Pectoral
        labels[(labels==13591)] = 1 # Left Major Pectoral
        labels[(labels==13592)] = 1 # RIght Major Pectoral
        labels[(labels==17943)] = 0 # Left Fat 
        labels[(labels==17944)] = 0 # Right Fat 
        
    elif num_classes == 5:
        labels[(labels!=13335)&(labels!=13336)&(labels!=13591)&(labels!=13592)&(labels!=17943)&(labels!=17944)] = 0
        labels[(labels==13335)] = 1 # Left Minor Pectoral
        labels[(labels==13336)] = 2 # Right Minor Pectoral
        labels[(labels==13591)] = 3 # Left Major Pectoral
        labels[(labels==13592)] = 4 # RIght Major Pectoral
        labels[(labels==17943)] = 0 # Left Fat 
        labels[(labels==17944)] = 0 # Right Fat
        
    elif num_classes == 3:
        labels[(labels!=13335)&(labels!=13336)&(labels!=13591)&(labels!=13592)&(labels!=17943)&(labels!=17944)] = 0
        labels[(labels==13335)] = 0 # Left Minor Pectoral
        labels[(labels==13336)] = 0 # Right Minor Pectoral
        labels[(labels==13591)] = 0 # Left Major Pectoral
        labels[(labels==13592)] = 0 # RIght Major Pectoral
        labels[(labels==17943)] = 1 # Left Fat 
        labels[(labels==17944)] = 2 # Right Fat 
        
    else:
        print('Introduce the follwoing number of classes: 2,3 or 5')
        ERROR = False

    if ERROR: labels_cat = (np.arange(num_classes) == labels[:,:,:,None])
        
    return labels_cat

In [ ]:
folder = '/Users/rmoreta/Documents/BWH Estancia 2016/ProjectData_clean/'
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'

dataKey_list = ['labels_nc2_pecs', 'labels_nc5_pecs', 'labels_nc3_fat']
num_classes_list = [2,5,3]
# dataKey_list = ['labels_nc2_pecs_borrar']
# num_classes_list = [5]

for ii in range(len(dataKey_list)):
    h5_keys = get_keys(h5_path)
    dataKey = dataKey_list[ii]
    num_classes = num_classes_list[ii]
    
    if dataKey in h5_keys:
        print('Key already existed. Now is deleted.')
        delete_dset(h5_path, dataKey)

    dset_size = get_shape_dset(h5_path, 'labels_raw')
    im_size = (0, dset_size[1], dset_size[2], num_classes)
    create_dset_no_data(h5_path, dataKey, im_size, 0, 'bool')
    len_dset = dset_size[0]

    widgets = ["Saving to h5: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
    pbar = progressbar.ProgressBar(maxval=len_dset, widgets=widgets).start()

    buff_size = 1000
    for i in np.arange(0, len_dset, buff_size):
        if i+buff_size > len_dset:
            idx = range(i, len_dset)
        else:
            idx = range(i, i+buff_size)
            
        data = get_dset_idx(h5_path, 'labels_raw', idx)
        data_labels = separate_labels(data, num_classes)
        add_to_dset(h5_path, dataKey, 0, data_labels)

        pbar.update(i)
    pbar.finish()

#### Saving Scan ID

In [ ]:
# path_csv = '/Volumes/GoogleDrive/Team Drives/GR.INV. BIIG-IGT/Projects/Pectoralis Segmentation/Data/pectoralis_copd_slices_clean_test_train.csv'
path_csv = '/Volumes/GoogleDrive/Team Drives/GR.INV. BIIG-IGT/Projects/Pectoralis Segmentation/Data/pectoralis_copd_slices_clean.csv'

csv_data = pd.read_csv(path_csv,',')
print(list(csv_data.keys()))

indexes = csv_data['bad_segmentatio'] == False

*Scan ID*

In [ ]:
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'

dataKey = 'scan_id'
h5_keys = get_keys(h5_path)
if dataKey in h5_keys:
    print('Key already existed. Now is deleted.')
    delete_dset(h5_path, dataKey)

data = csv_data['scan_id'][indexes].values
dataShape = data.shape    

create_dset(h5_path, dataKey, dataShape, 0, 'f', data)

*code* ((NEEDS TO BE FIX)

In [ ]:
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'

dataKey = 'code'
h5_keys = get_keys(h5_path)
if dataKey in h5_keys:
    print('Key already existed. Now is deleted.')
    delete_dset(h5_path, dataKey)

data = np.string_(csv_data[dataKey][indexes].values)
# dataShape = (10006,)

create_dset(h5_path, dataKey, (10006,), 0, 'S10', data)

*'slice', 'container_id', 'container_slice'*

In [ ]:
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'

dataKey = 'slice_&_container'
h5_keys = get_keys(h5_path)
if dataKey in h5_keys:
    print('Key already existed. Now is deleted.')
    delete_dset(h5_path, dataKey)

csv_keys = ['slice', 'container_id', 'container_slice']
data = []
for csv_key in csv_keys:
    aux = csv_data[csv_key][indexes].values
    data.append(aux)
data = np.swapaxes(np.array(data),0,1)
dataShape = data.shape    

create_dset(h5_path, dataKey, dataShape, 0, 'f', data)

------

------

### Check if it has been saved correctly

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt

In [ ]:
folder = '/Users/rmoreta/Documents/BWH Estancia 2016/ProjectData_clean/'
h5_path = '/data/DNNData/rm196/PectoralisSegmentation/Data/myfinaltestfile.hdf5'

In [ ]:
keys = get_keys(h5_path)

In [ ]:
num_files = [6] # 7 nrrd files in total

CTSlices = []
labels = []
# Reading CT

for i in num_files:
    if i == num_files[0]:
        # CTSlices
        file_name_CT = 'Cont_' + str(i+1) + '_clean.nrrd'#'_clean.nrrd'
        image_nrrd = sitk.ReadImage(folder + file_name_CT)
        CTSlices = sitk.GetArrayFromImage(image_nrrd)
        # Labels
        file_name_label = 'Cont_lm_' + str(i+1) + '_clean.nrrd'
        image_nrrd = sitk.ReadImage(folder + file_name_label)
        labels = sitk.GetArrayFromImage(image_nrrd)
    else:
        # CTSlices
        file_name_CT = 'Cont_' + str(i+1) + '_clean.nrrd'
        image_nrrd = sitk.ReadImage(folder + file_name_CT)
        CTSlices_aux = sitk.GetArrayFromImage(image_nrrd)
        CTSlices = np.concatenate((CTSlices, CTSlices_aux), axis = 0)
        # Labels
        file_name_label = 'Cont_lm_' + str(i+1) + '_clean.nrrd'
        image_nrrd = sitk.ReadImage(folder + file_name_label)
        labels_aux = sitk.GetArrayFromImage(image_nrrd)
        labels = np.concatenate((labels, labels_aux), axis = 0)
        
        
print('CTSlices Shape: ')
print(CTSlices.shape)

print('Labels Shape: ')
print(labels.shape)

num_imgs_list = [1455, 1449, 1435, 1451, 1435, 1444, 1337]
num_imgs_cont = CTSlices.shape[0]

fr = int(np.sum(num_imgs_list[0:i]))
to = fr + num_imgs_cont 

In [ ]:
def plot_images_one_channel(images):
    num_imgs = len(images)
    f, a = plt.subplots(1, num_imgs, figsize=(20, 20))
    for i in range(num_imgs):
        a[i].imshow(images[i],cmap='Greys_r')
    f.show()
    plt.draw()

#### Raw

In [ ]:
i = np.random.randint(num_imgs_cont)
i_h5 = (i + int(np.sum(num_imgs_list[0:num_files[0]])))

raw_img = CTSlices[i]
raw_label = labels[i]

raw_img_h5 = get_dset_idx(h5_path, 'images_raw', i_h5)
raw_labels_h5 = get_dset_idx(h5_path, 'labels_raw', i_h5)

plot_images_one_channel([raw_img, raw_img_h5])
plot_images_one_channel([raw_label, raw_labels_h5])

if np.array_equal(raw_img, raw_img_h5):
    print('Raw images are the same')
    
if np.array_equal(raw_label, raw_labels_h5):
    print('Raw labels are the same')

#### Norm

In [ ]:
numpy_array = CTSlices.astype(np.float32)
numpy_array[numpy_array < -1024] = -1024
numpy_array[numpy_array > 1500] = 1500
numpy_array = (numpy_array + 1024) / (1500 + 1024) - 0.5

CTSlices_new = numpy_array
print(CTSlices_new.shape)

In [ ]:
i = np.random.randint(num_imgs_cont)
i_h5 = (i + int(np.sum(num_imgs_list[0:num_files[0]])))
print('image: ', i, i_h5)

norm_img = CTSlices_new[i]

norm_img_h5 = get_dset_idx(h5_path, 'images_norm', i_h5)

plot_images_one_channel([norm_img, norm_img_h5])

if np.array_equal(norm_img, norm_img_h5):
    print('Raw images are the same')
    

#### Labels number classes

In [ ]:
def separate_labels(labels, num_classes):
    ERROR = False
    if num_classes == 2:
        labels[(labels!=13335)&(labels!=13336)&(labels!=13591)&(labels!=13592)&(labels!=17943)&(labels!=17944)] = 0
        labels[(labels==13335)] = 1 # Left Minor Pectoral
        labels[(labels==13336)] = 1 # Right Minor Pectoral
        labels[(labels==13591)] = 1 # Left Major Pectoral
        labels[(labels==13592)] = 1 # RIght Major Pectoral
        labels[(labels==17943)] = 0 # Left Fat 
        labels[(labels==17944)] = 0 # Right Fat 
        
    elif num_classes == 5:
        labels[(labels!=13335)&(labels!=13336)&(labels!=13591)&(labels!=13592)&(labels!=17943)&(labels!=17944)] = 0
        labels[(labels==13335)] = 1 # Left Minor Pectoral
        labels[(labels==13336)] = 2 # Right Minor Pectoral
        labels[(labels==13591)] = 3 # Left Major Pectoral
        labels[(labels==13592)] = 4 # RIght Major Pectoral
        labels[(labels==17943)] = 0 # Left Fat 
        labels[(labels==17944)] = 0 # Right Fat
        
    elif num_classes == 3:
        labels[(labels!=13335)&(labels!=13336)&(labels!=13591)&(labels!=13592)&(labels!=17943)&(labels!=17944)] = 0
        labels[(labels==13335)] = 0 # Left Minor Pectoral
        labels[(labels==13336)] = 0 # Right Minor Pectoral
        labels[(labels==13591)] = 0 # Left Major Pectoral
        labels[(labels==13592)] = 0 # RIght Major Pectoral
        labels[(labels==17943)] = 1 # Left Fat 
        labels[(labels==17944)] = 2 # Right Fat 
        
    else:
        print('Introduce the follwoing number of classes: 2,3 or 5')
        ERROR = True

    if not(ERROR): labels_cat = (np.arange(num_classes) == labels[:,:,:,None])
        
    return labels_cat

In [ ]:
def plot_lables_nc(labels_nc, num_classes, var):
    if var == 0: labels_nc = labels_nc[0]
    
    f, a = plt.subplots(1, num_classes, figsize=(20, 20))
    for i in range(num_classes):
        a[i].imshow(labels_nc[:,:,i],cmap='Greys_r')
    f.show()
    plt.draw()

In [ ]:
i = np.random.randint(num_imgs_cont)
i_h5 = (i + int(np.sum(num_imgs_list[0:num_files[0]])))
print('image: ', i, i_h5)

dataKey_list = ['labels_nc2_pecs', 'labels_nc5_pecs', 'labels_nc3_fat']
num_classes_list = [2,5,3]

raw_img = CTSlices[i]
plot_images_one_channel([raw_img, raw_img])

for dataKey, nc in zip(dataKey_list, num_classes_list):
    
    image_nrrd = sitk.ReadImage(folder + file_name_label)
    labels = sitk.GetArrayFromImage(image_nrrd)
    
    labels_nc = separate_labels(labels[i:i+1], nc)

    labels_nc_h5 = get_dset_idx(h5_path, dataKey, i_h5)

    print(labels_nc.shape)
    if np.array_equal(labels_nc[0], labels_nc_h5):
        print('Raw images are the same')
    
    plot_lables_nc(labels_nc, nc, 0)
    plot_lables_nc(labels_nc_h5, nc, 1)